#Project#

In [ ]:
print("hi")

hi


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow opencv-python sklearn #install tensorflow

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
import pandas as pd  # For data manipulation
import numpy as np
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#Assign the dataset path to a variable
data_directory = '/content/drive/MyDrive/SDGP_Project/Dataset'

import os
import cv2

#face shape catogaries
categories = os.listdir(data_directory)

# references - https://towardsdatascience.com/implementing-k-nearest-neighbors-with-scikit-learn-9e4858e231ea

# Initialize lists to store images and labels
images = []
labels = []

# Iterate through each category of images
for category in categories:
    category_path = os.path.join(data_directory, category)
    for image_path in os.listdir(category_path):
        image = cv2.imread(os.path.join(category_path, image_path)) # Read the image using OpenCV
        # Resize the image to a consistent size
        image = cv2.resize(image, (224, 224))  # Adjust size as needed

        # Append the image and its corresponding label to the lists
        images.append(image)
        labels.append(category)

# Convert images to NumPy arrays
images = np.array(images)
images

array([[[[126, 116,  69],
         [124, 114,  67],
         [119, 110,  66],
         ...,
         [189, 213, 231],
         [189, 213, 231],
         [189, 213, 231]],

        [[120, 110,  63],
         [119, 109,  62],
         [114, 105,  61],
         ...,
         [189, 213, 231],
         [189, 213, 231],
         [188, 212, 230]],

        [[112, 102,  55],
         [111, 101,  54],
         [107,  98,  54],
         ...,
         [189, 213, 231],
         [188, 212, 230],
         [188, 212, 230]],

        ...,

        [[189, 201, 189],
         [189, 201, 189],
         [189, 201, 189],
         ...,
         [ 67,  63,  62],
         [ 68,  64,  63],
         [ 67,  63,  62]],

        [[188, 201, 187],
         [188, 201, 187],
         [189, 202, 188],
         ...,
         [ 65,  63,  62],
         [ 67,  65,  64],
         [ 66,  64,  63]],

        [[187, 198, 182],
         [187, 198, 182],
         [188, 199, 183],
         ...,
         [ 64,  65,  63],
        